# Database Utilities Demo

This notebook demonstrates how to use the enhanced DatabaseHandler for common database operations.

In [1]:
from crypto_trading.db import DatabaseHandler

# Initialize handler with default database path
db = DatabaseHandler()

## List Available Tables

In [2]:
# List all tables
all_tables = db.list_tables()
display(all_tables)

# List tables in specific schema
amberdata_tables = db.list_tables(schema='amberdata')
display(amberdata_tables)

,schema,table,type
0,amberdata,exchange_reference,BASE TABLE
1,amberdata,ohlcv_info_futures,BASE TABLE
2,amberdata,ohlcv_perps_1d,BASE TABLE
3,main,spot_ohlc_1h,BASE TABLE


,schema,table,type
0,amberdata,exchange_reference,BASE TABLE
1,amberdata,ohlcv_info_futures,BASE TABLE
2,amberdata,ohlcv_perps_1d,BASE TABLE


## Examine Table Schema

In [3]:
# Get schema for OHLCV table
schema = db.get_table_schema('amberdata', 'ohlcv_perps_1d')
display(schema)

,column_name,data_type,is_nullable,column_default
0,instrument,VARCHAR,YES,None
1,open,DOUBLE,YES,None
2,high,DOUBLE,YES,None
3,low,DOUBLE,YES,None
4,close,DOUBLE,YES,None
5,volume,DOUBLE,YES,None
6,exchange,VARCHAR,YES,None
7,datetime,TIMESTAMP_NS,YES,None


## Get Table Statistics

In [4]:
# Get info about OHLCV table
info = db.get_table_info('amberdata', 'ohlcv_perps_1d')
print(f"Row Count: {info['row_count']:,}")
print(f"Column Count: {info['column_count']}")
print("\nColumn Types:")
for type_name, count in info['column_types'].items():
    print(f"  {type_name}: {count}")

Row Count: 475,637
Column Count: 8

Column Types:
  DOUBLE: 5
  VARCHAR: 2
  TIMESTAMP_NS: 1


## Query Data

In [5]:
# Simple query example
query = """
SELECT 
    exchange,
    instrument,
    COUNT(*) as record_count,
    MIN(datetime) as start_date,
    MAX(datetime) as end_date
FROM amberdata.ohlcv_perps_1d
GROUP BY exchange, instrument
ORDER BY exchange, instrument
LIMIT 5
"""

df = db.query_to_df(query)
display(df)

,exchange,instrument,record_count,start_date,end_date
0,binance,1000000MOGUSDT,71,2024-11-07,2025-01-16
1,binance,1000BONKUSDC,260,2024-05-02,2025-01-16
2,binance,1000BONKUSDT,422,2023-11-22,2025-01-16
3,binance,1000BTTCUSDT,76,2022-01-26,2022-04-11
4,binance,1000CATUSDT,88,2024-10-21,2025-01-16


## Analyze Data Coverage

In [6]:
# Get data coverage by month
coverage = db.get_date_range_stats(
    schema='amberdata',
    table='ohlcv_perps_1d',
    timestamp_col='datetime',
    grouping='month'
)
display(coverage)

,period,record_count
0,2018-11-01,17
1,2018-12-01,31
2,2019-01-01,38
3,2019-02-01,56
4,2019-03-01,62
...,...,...
70,2024-09-01,15475
71,2024-10-01,16678
72,2024-11-01,21418
73,2024-12-01,28567


## Get Latest Data

In [7]:
# Get latest timestamp
latest = db.get_latest_timestamp(
    schema='amberdata',
    table='ohlcv_perps_1d',
    timestamp_col='datetime'
)
print(f"Latest data point: {latest}")

Latest data point: 2025-01-16 00:00:00
